In [ ]:
"""
# 👥 Collaborative Filtering Model

Build and evaluate collaborative filtering with SVD
"""

# ## 1. Setup
import pandas as pd
import sys
sys.path.append('..')

from src.models.collaborative import CollaborativeFilteringRecommender
from config import ANIME_CLEANED, RATING_SAMPLED, COLLABORATIVE_MODEL

# ## 2. Load Data
anime_df = pd.read_csv(ANIME_CLEANED)
ratings_df = pd.read_csv(RATING_SAMPLED)

print(f"Anime: {anime_df.shape}")
print(f"Ratings: {ratings_df.shape}")

# ## 3. Initialize Model
model = CollaborativeFilteringRecommender(
    algorithm='SVD',
    n_factors=100,
    n_epochs=20
)

# ## 4. Prepare Data
print("\n=== Preparing Data ===")
trainset, testset = model.prepare_data(ratings_df, anime_df, test_size=0.2)

# ## 5. Train Model
print("\n=== Training Collaborative Model ===")
model.fit(trainset)

# ## 6. Evaluate
print("\n=== Evaluating Model ===")
metrics = model.evaluate(testset)

print(f"\nTest RMSE: {metrics['RMSE']:.4f}")
print(f"Test MAE: {metrics['MAE']:.4f}")

# ## 7. Test Recommendations
print("\n=== Testing Recommendations ===")

test_user_id = ratings_df.iloc[0]['user_id']
recommendations = model.recommend_for_user(test_user_id, top_n=10)

print(f"\nRecommendations for User {test_user_id}:")
print(recommendations[['Name', 'Score', 'Genres', 'predicted_rating']])

# ## 8. Save Model
model.save_model(COLLABORATIVE_MODEL)
print(f"\n✅ Model saved to {COLLABORATIVE_MODEL}")

# ## 9. Analyze Results
import matplotlib.pyplot as plt

# Plot predictions vs actual
from surprise import accuracy

predictions = model.model.test(testset)
pred_ratings = [pred.est for pred in predictions]
true_ratings = [pred.r_ui for pred in predictions]

plt.figure(figsize=(10, 6))
plt.scatter(true_ratings, pred_ratings, alpha=0.3)
plt.plot([1, 10], [1, 10], 'r--')
plt.xlabel('True Rating')
plt.ylabel('Predicted Rating')
plt.title('Prediction vs Actual')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()